In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

#!bash /Users/derekdewald/scripts/daily_shell_job.sh

Need to Add Update of Learning File

In [ ]:
from daily_processes import daily_test

#review_test_results()
daily_test()
review_test_results()

#df = pd.read_csv('/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv')
#df

In [ ]:
words = []

from data_d_strings import gpt_question
gpt_question(words)